# LaTeX Support

Note that LaTeX formulas are not fully supported when exporting to PNG/PDF.

In [1]:
import numpy as np
import pandas as pd

from lets_plot import *

In [2]:
LetsPlot.setup_html()

## Power Degree

In [3]:
data1 = {'x': list(range(-10, 11))}
ggplot() + \
    geom_function(aes(x='x'), data=data1, fun=lambda x: x**3 - 100 * x) + \
    ggtitle("Graph of the \( y = x^3 - 100 x \)") + \
    theme(text=element_text(family="Times New Roman"), \
          plot_title=element_text(size=20, face='bold'))

## Subscript

In [4]:
raw_df = pd.read_csv("https://raw.githubusercontent.com/JetBrains/lets-plot-docs/refs/heads/master/data/co-emissions-per-capita.csv")
raw_df.columns = ["country", "code", "year", "co2"]
df = raw_df[raw_df["country"].isin(["World", "United States", "United Kingdom", "China", "India"])]

ggplot(df, aes("year", "co2", color="country")) + \
    geom_line(tooltips=layer_tooltips().title("@year\nin tonnes per person").format("year", "d")
                                       .line("@country: @co2").format("@co2", "{.1f} t")) + \
    scale_x_continuous(name="", format="d") + \
    scale_y_continuous(name="", format="{d} t", expand=[0, 0]) + \
    labs("Per capita \( CO_2 \) emissions",
         "Carbon dioxide (\( CO_2 \)) emissions from fossil fuels and industry. "
         "Land-use change is not included.",
         "<a href=\"https://ourworldindata.org/explorers/co2?facet=none&hideControls=false&Gas+or+Warming=CO%E2%82%82&Accounting=Production-based&Fuel+or+Land+Use+Change=All+fossil+emissions&Count=Per+capita&country=CHN~USA~IND~GBR~OWID_WRL\">Original plot</a>") + \
    ggsize(1000, 400) + \
    theme(text=element_text(family="Playfair Display"), \
          plot_title=element_text(size=20, face='bold'))

## Greek Letters and Other Special Symbols

In [5]:
ggplot() + \
    geom_text(x=0, label=r"\( e^{i \cdot \pi} = -1 \)", \
              size=70, family="Computer Modern Math", fontface='italic') + \
    theme_void()

## Place Your Formulas Anywhere!

In [6]:
data2 = {
    "x": [r"\( \Omega_{\rho + 1} \)"],
    r"\(\delta\)": [1],
    "f": [r"\( A \neq B^2 \)"],
}

ggplot(data2) + \
    geom_bar(aes("x", r"\(\delta\)", fill="x"), stat='identity', \
             labels=layer_labels().line("@f"), \
             tooltips=layer_tooltips().line("@f")) + \
    scale_fill_manual(["#6a3d9a"], name=r"\(\Sigma\)") + \
    xlab(r"\(\sigma\)") + \
    theme(label_text=element_text(size=20))

## Curly Braces Escaping

Let's look at the following example:

In [7]:
def get_data(n, seed=42):
    np.random.seed(seed)
    return {
        "x": np.random.choice([
            r"\( x^a+1 \)",
            r"\( x^{a \cdot b}+1 \)",
            r"\( x^{a \cdot b^c}+1 \)",
            r"\( x^{a \cdot b^{c - d}}+1 \)",
        ], size=n, p=[.5, .25, .2, .05]),
    }

data3 = get_data(50)

In [8]:
ggplot(data3, aes(x=as_discrete("x", order_by='..count..'))) + geom_bar()

As in the other bars, the unit in the last bar must be at the same level as x. But it isn't.

Let's find out why this is so. \
In this example, the formulas are passed to the plot as a list of strings in the dataset. The Lets-Plot concludes that the default formatting for strings, `"{}"` (an empty placeholder), should be applied to them. This formatting causes the `}` bracket to be escaped. Therefore, to get `}}`, we must use `}}}}` (see [Formatting](https://lets-plot.org/python/pages/formats.html#string-template)):

In [9]:
data3["x"] = [f.replace("}}", "}}}}") for f in data3["x"]] # this accounts for curly braces escaping

ggplot(data3, aes(x=as_discrete("x", order_by='..count..'))) + geom_bar()

Or, if that works for you, you can just disable the default string formatting:

In [10]:
ggplot(get_data(50), aes(x=as_discrete("x", order_by='..count..'))) + geom_bar() + \
    scale_x_discrete(format="") # this disables the default formatting of the x-axis ticks